# Databricks Storage Mounts

**Benefits**
- Access data without requiring credentials
- Access files using file semantics instead of storage URLs
- Stores files to object storage realizing Azure storage benefits

**Workflow**
- Mounts ADLS Gen2 to DBFS Using Service Principal
- Access DBFS from Databricks Notebooks

In [0]:
# Set Variables

client_id = ""
tenant_id = ""
secret_value = ""

In [0]:
# Set Variables Using Secret Scope

client_id = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
tenant_id = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
secret_value = dbutils.secrets.get(scope=<secret_scope>, key=<secret_scope_key>)
service_credential = dbutils.secrets.get(scope="<secret_scope>",key="<service_scope_key>")

In [0]:
# Set Spark Config

spark.conf.set("fs.azure.account.auth.type.<storage_account_namet>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage_account_namet>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage_account_namet>.dfs.core.windows.net", "<application_id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage_account_name>.dfs.core.windows.net", <service_credential_client_secret>)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage_account_namet>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory_id>/oauth2/token")